<a href="https://colab.research.google.com/github/KNU-BrainAI-Capstone2022/ITS/blob/main/review_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 6.7 MB/s 
     |████████████████████████████████| 163 kB 57.9 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer, BertForSequenceClassification

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [4]:
path = '/content/drive/MyDrive/fake reviews dataset.csv'

df = pd.read_csv(path)
print(df.shape)
df.head()

(40432, 4)


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [5]:
df['division'] = np.select([df.rating > 3], [1], default=0)
df[:5]

,category,rating,label,text_,division
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",1
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",1
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,1
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",0
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,1


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 한국어 및 다른 언어시 변경 필요

# example_text = 'I will watch Memento tonight'
# bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
#                        truncation=True, return_tensors="pt")

# # padding : to pad each sequence to the maximum length that you specify
# # max_length : the maximum length of each sequence.
# #              For our actual dataset we will use 512, which is the maximum length of a sequence allowed for BERT.
# # truncation : if True, then the tokens in each sequence that exceed the maximum length will be truncated.
# # return_tensors : the type of tensors that will be returned. 
# #                  Pytorch: pt. 
# #                  Tensorflow: tf


# print(bert_input['input_ids'])
# # >>>tensor([[  101,   146,  1209,  2824,  2508, 26173,  3568,   102,     0,     0]])
# # tokenizer.decode(bert_input.input_ids[0])
# # >>> [CLS] I will watch Memento tonight [SEP] [PAD] [PAD]
# # BertTokenizer가 알아서 다 해줌
# print(bert_input['token_type_ids'])
# # >>>tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
# print(bert_input['attention_mask'])
# # >>>tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.divisions = [division for division in df['division']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text_']]
    def classes(self):
        return self.divisions

    def __len__(self):
        return len(self.divisions)

    def get_batch_divisions(self, idx):
        # Fetch a batch of labels
        return np.array(self.divisions[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_divisions(idx)

        return batch_texts, batch_y

df[:5]

,category,rating,label,text_,division
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",1
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",1
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,1
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",0
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,1


In [8]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.7*len(df)), int(.9*len(df))]) # train: 70, validation: 20, test: 10

print(len(df_train),len(df_val), len(df_test))

28302 8086 4044


In [9]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 다른 언어의 데이터면 다른거 쓰기
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # _: embedding vectors of all of the tokens in a sequence
        # pooled_output: embedding vector of [CLS] token, 분류 문제에선 이것만 써도 충분
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [10]:
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs, cuda_switch):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr= learning_rate)

    if cuda_switch:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_division in tqdm(train_dataloader):

                train_division = train_division.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_division)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_division).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_division in val_dataloader:

                    val_division = val_division.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_division)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_division).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 2 
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS, use_cuda)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 14151/14151 [50:09<00:00,  4.70it/s]


Epochs: 1 | Train Loss:  0.203                 | Train Accuracy:  0.827                 | Val Loss:  0.180                 | Val Accuracy:  0.852


100%|██████████| 14151/14151 [50:17<00:00,  4.69it/s]


Epochs: 2 | Train Loss:  0.164                 | Train Accuracy:  0.866                 | Val Loss:  0.170                 | Val Accuracy:  0.858


In [11]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_division in test_dataloader:

              test_division = test_division.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_division).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.845
